In [1]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCN, GAT, Transformer
from sklearn.neighbors import LocalOutlierFactor
import pandas as pd
import numpy as np

ImportError: cannot import name 'Transformer' from 'torch_geometric.nn' (/home/oleg/Localdev/Apertus/Mine-the-Gap/.venv/lib/python3.13/site-packages/torch_geometric/nn/__init__.py)

In [ ]:
#Load the dataset
dataset = pd.read_csv('./data/edge_events.csv')
# Extract unique nodes and create mapping dictionaries
nodes = pd.unique(dataset[["src", "dst"]].to_numpy().ravel())
node2idx = {n: i for i, n in enumerate(nodes)}
idx2node = {i: n for n, i in node2idx.items()}

# Map source and destination nodes to integer IDs
src = dataset["src"].map(node2idx).to_numpy()
dst = dataset["dst"].map(node2idx).to_numpy()

# Extract timestamps and labels
t = dataset["timestamp"].astype(int).to_numpy()
msg = pd.get_dummies(dataset["label"]).to_numpy()

In [ ]:


# Load graph data
df = pd.read_csv('graph_data.csv')
adj = df.adj.toarray()
x = df.x
y = df.y

# Define graph embedding model
model = GCN(x, adj, num_layers=2, hidden_dim=64).to('cuda')

# Define anomaly detection model
X = torch.tensor(x)
y_pred = np.zeros_like(x)
scores = np.zeros_like(x)
outlier_scores = np.zeros_like(x)

for i, (data) in enumerate(DataLoader(Data(x=x, adj=adj, y=y), batch_size=len(x))):
    out = model(data.x)
    scores[i*len(x):(i+1)*len(x)] = out

# Define anomaly detection algorithm
lof = LocalOutlierFactor(n_neighbors=20, novelty=True)
scores = lof.fit_predict(scores)

# Normalize scores to [0, 1]
scores = np.clip(scores, 0, 1)

# Output anomaly scores
anomaly_scores = pd.DataFrame({'node_id': np.arange(len(x)), 'anomaly_score': scores})

# Define LLM interface
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

def get_anomalies(question):
    question = tokenizer.encode(question, return_tensors='pt')
    inputs = {'input_ids': question['input_ids']}
    outputs = model.generate(**inputs)
    anomaly_scores = anomaly_scores.to_numpy().tolist()

    return anomaly_scores

# Example usage
question = "What are the top 5 most anomalous nodes in the graph?"
out = get_anomalies(question)
print(out)